In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import unix_timestamp, col, lit, from_unixtime, expr, sum as _sum
from pyspark.sql import functions as F
from datetime import datetime


spark = SparkSession.builder \
    .appName("MongoDB Aggregation Query") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/vehicle") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/vehicle") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.4.0")\
    .getOrCreate()

# Updated Schema
schema = StructType([
    StructField("_id", StringType(), True),
    StructField("link", StringType(), True),
    StructField("time", LongType(), True),
    StructField("vcount", LongType(), True),
    StructField("vspeed", DoubleType(), True)
])

raw_data_schema = StructType([
    StructField("_id", StringType(), True),
    StructField("destination", StringType(), True),
    StructField("link", StringType(), True),
    StructField("origin", StringType(), True),
    StructField("position", StructType([ 
        StructField("x", DoubleType(), True),
        StructField("y", DoubleType(), True)
    ]), True),
    StructField("spacing", DoubleType(), True),
    StructField("speed", DoubleType(), True),
    StructField("time", StringType(), True),  
    StructField("vehicle_name", StringType(), True),
    StructField("total_distance", DoubleType(), True)  # προστέθηκε για να το εμφανίσουμε μετα
])

def query_least_traffic(spark, low, great):
    df = spark.read.format("mongodb")\
                   .option("database", "vehicle")\
                   .option("collection", "vehicle_position")\
                   .schema(schema)\
                   .load()
    
    df.printSchema()
    df.show(5)

    pipeline = f'''[
    {{"$match": {{"time": {{"$gt": {low}, "$lt": {great}}}}}}},
    {{"$sort": {{"vcount": 1}}}}
     ]'''
    
    df_pipeline = spark.read.format("mongodb")\
                            .option("database", "vehicle")\
                            .option("collection", "vehicle_position")\
                            .option("aggregation.pipeline", pipeline)\
                            .schema(schema)\
                            .load()
    

    df_pipeline.show()

def query_max_speed(spark, low, great):
    
    df = spark.read.format("mongodb")\
                   .option("database", "vehicle")\
                   .option("collection", "vehicle_position")\
                   .schema(schema)\
                   .load()
    
    df.printSchema()
    df.show(5)

   
    pipeline = f'''
    [
        {{"$match": {{"time": {{"$gt": {low}, "$lt": {great}}}}}}},
        {{"$sort": {{"vspeed": -1}}}}
    ]
    '''

    df_pipeline = spark.read.format("mongodb")\
                            .option("database", "vehicle")\
                            .option("collection", "vehicle_position")\
                            .option("aggregation.pipeline", pipeline)\
                            .schema(schema)\
                            .load()
    
  
    df_pipeline.show()

 #τροπος αναπαραστασης timestamp στην mongoDB 
def unix_to_iso(unix_time):
    """Convert UNIX timestamp to ISO 8601 format."""
    return datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%SZ')

def query_max_distance(spark, low, great):
      
        df = spark.read.format("mongodb")\
                       .option("database", "vehicle")\
                       .option("collection", "raw_data")\
                       .schema(raw_data_schema)\
                       .load()

        # Debug: print schema and sample data
        print("Schema and sample data for max distance query:")
        df.printSchema()
        df.show(5)
        # μετατραπη της είσοδου σε κατάλληλα timestamp στο τροπο αναπαράστασης της mongoDb
        min_time = df.selectExpr("min(time) as min_time").collect()[0]["min_time"]
        print(f"Minimum time (start of simulation): {min_time}")
        min_time_unix = df.select(unix_timestamp(lit(min_time), "yyyy-MM-dd'T'HH:mm:ss.SSSX").alias("min_time_unix")).collect()[0]["min_time_unix"]
        low_unix = min_time_unix + low
        great_unix = min_time_unix + great
        low_iso = unix_to_iso(low_unix)
        great_iso = unix_to_iso(great_unix)

       # pipeline για το query 
        pipeline = f'''
        [ 
            {{"$match": {{"time": {{"$gte": ISODate("{low_iso}"), "$lt": ISODate("{great_iso}")}}}}}},
            {{"$group": {{"total_distance": {{"$sum": "$spacing"}},"_id": "$vehicle_name" }}}},
            {{"$sort": {{"total_distance": -1}}}}
           
        ]'''
        df_pipeline = spark.read.format("mongodb")\
                                .option("database", "vehicle")\
                                .option("collection", "raw_data")\
                                .option("aggregation.pipeline", pipeline)\
                                .schema(raw_data_schema)\
                                .load()

        max_distance_row = df_pipeline.agg({"total_distance": "max"}).collect()[0]
        max_distance = max_distance_row["max(total_distance)"]

        df_result = df_pipeline.filter(df_pipeline.total_distance == max_distance)
        df_result = df_result.select("_id", "total_distance")

        
        print("Vehicle with the maximum total distance:")
        results = df_result.collect()
        for row in results:
            print(f"Vehicle ID: {row['_id']}, Total Distance: {row['total_distance']}")

        return df_result

   
def main():
    #menu
    choice = int(input("Choose Query \n (1 for least traffic, 2 for max speed, 3 for max distance): "))
    
    
    low = int(input("Enter low time as long: "))
    great = int(input("Enter great time as long: "))

    if choice == 1:
        query_least_traffic(spark, low, great)
    elif choice == 2:
        query_max_speed(spark, low, great)
    elif choice == 3:
        query_max_distance(spark, low, great)
    else:
        print("Invalid choice")

#εναρξη
main()



Choose Query 
 (1 for least traffic, 2 for max speed, 3 for max distance):  3
Enter low time as long:  4
Enter great time as long:  8


Schema and sample data for max distance query:
root
 |-- _id: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- link: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- position: struct (nullable = true)
 |    |-- x: double (nullable = true)
 |    |-- y: double (nullable = true)
 |-- spacing: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- time: string (nullable = true)
 |-- vehicle_name: string (nullable = true)
 |-- total_distance: double (nullable = true)

+--------------------+-----------+--------+------+--------------+-------+-----+--------------------+------------+--------------+
|                 _id|destination|    link|origin|      position|spacing|speed|                time|vehicle_name|total_distance|
+--------------------+-----------+--------+------+--------------+-------+-----+--------------------+------------+--------------+
|66cdd7389289aa37d...|         N1|S4_to_I4|    S4|{300.0, 100.0}|  100.0| 30.0|2024-08-24T